In [2]:
import numpy as np
def f(x: list):
    return 105*(x[0]**2) - 2*x[0]*x[1] + 105*(x[1]**2) - 96*x[0] + 25*x[1] - 13

In [89]:
def generate_vector():
    return np.random.uniform(-1, 1, 2)

def calculate_vector(vector: list):
    return np.sqrt(vector[0]**2+vector[1]**2)

In [1]:
class RandomSearch:
    x: list
    eps: float
    alpha: float
    gamma: float
    j: int
    M: int
    vector: list
    # шаг 1
    def __init__(self, x, eps, alpha, gamma, M):
        self.x = x
        self.eps = eps
        self.alpha = alpha
        self.gamma = gamma
        self.M = M
    
    # функция для старта алгоритма
    def start(self):
        self.__step_2()

    def __step_2(self):
        # полагаем счётчик неудачных попыток = 1
        self.j = 1
        # переходим к шагу 3
        self.__step_3()

    def __step_3(self):
        # получаем реализацию случайного вектора
        self.vector = generate_vector()
        # вычисляем норму вектора
        self.calculated_vector = calculate_vector(self.vector)
        # переходим к шагу 4
        self.__step_4()
    
    def __step_4(self):
        # ищем пробную точку y
        y = [self.x[0] + self.alpha * self.vector[0]/self.calculated_vector , self.x[1] + self.alpha * self.vector[1]/self.calculated_vector]
        # вычисляем f(y) и переходим к шагу 5
        self.__step_5(y=y, fy=f(y))
    
    def __step_5(self, y, fy):
        # если f(y) < f(x)
        if fy < f(self.x):
            # положим x = y
            self.x = y
            # к шагу 4
            self.__step_4()
        # иначе к шагу 6
        else: self.__step_6()

    def __step_6(self):
        # увеличиваем счётчик неудачных попыток
        self.j = self.j + 1
        # если j <= M
        if self.j <= self.M:
            # переходим к шагу 3
            self.__step_3()
        # иначе к шагу 7
        else: self.__step_7()
    
    def __step_7(self):
        print("step7-alpha", self.alpha)
        # если альфа < eps, то завершаем поиск и осуществляем вывод
        if self.alpha < self.eps:
            print("x*=",self.x)
            print("f*=",f(self.x))
        else: 
            # иначе уменьшаем альфа и обновляем счётчик
            self.alpha = self.alpha / self.gamma
            self.__step_2()

In [212]:
x0 = [-13, 0]
eps = 0.005
alpha = 0.2
gamma = 2
j = 1
M = 6
alg = RandomSearch(x=x0, eps=eps, alpha=alpha, gamma=gamma, M=M)
alg.start()

step7-alpha 0.2
step7-alpha 0.1
step7-alpha 0.05
step7-alpha 0.025
step7-alpha 0.0125
step7-alpha 0.00625
step7-alpha 0.003125
x*= [0.45704210680295576, -0.1157305469265379]
f*= -36.32400853757567


In [7]:
class ResearchSearch: 
    x: list
    vector: list
    j: int
    e_j = [[1, 0], [0, 1]]
    def __init__(self, x, vector):
        self.x = x
        self.vector = vector

    
    def start(self):
        self.j = 1
        self.__step_2()

    def __step_2(self):
        # осуществляем пробный шаг
        y = [self.x[0] - self.vector[self.j-1] * self.e_j[self.j-1][0], self.x[1] - self.vector[self.j-1] * self.e_j[self.j-1][1]]
        # если f(x`) <= f(y)
        if f(self.x) <= f(y):
            # то переходим к шагу 3
            self.__step_3()
        # иначе к шагу 4
        else: self.__step_4(y)

    def __step_3(self):
        # делаем пробный шаг 
        y = [self.x[0] + self.vector[self.j-1] * self.e_j[self.j-1][0], self.x[1] + self.vector[self.j-1] * self.e_j[self.j-1][1]]
        # если f(x`) <= f(y)
        if f(self.x) <= f(y):
            # то переходим к шагу 5
            self.__step_5()
        # иначе к шагу 4
        else: self.__step_4(y)
    
    def __step_4(self, y):
        # полагаем x` = y
        self.x = y
        # переходим к шагу 5
        self.__step_5()

    def __step_5(self):
        # полагаем j = j +1
        self.j = self.j + 1
        # если j <= n
        if self.j <= len(self.vector):
            # то перейти к шагу 2
            self.__step_2()
        # иначе поиск окончен

In [8]:

class HookJeeves:
    x: list
    eps: float
    gamma: float
    vector: list
    calculated_vector: float
    def __init__(self, x, vector, eps, gamma):
        self.x = x
        self.eps = eps
        self.gamma = gamma
        self.vector = vector
        self.calculated_vector = calculate_vector(vector)    
    def start(self):
        self.__step_2()

    def __step_2(self):
        # выполняем исследующий покоординатный поиск из x0, чтобы найти x0`
        search_alg = ResearchSearch(self.x, self.vector)
        search_alg.start()
        calculated_x = search_alg.x
        # если поиск вернул не x0
        if (calculated_x != self.x):
            # то переходим к шагу 4
            self.__step_4(calculated_x)
        # иначе переходим к проверке окончания поиска
        else: self.__step_3()

    def __step_3(self):
        # если достигнут минимум, то возвращаем результат
        if (self.calculated_vector < self.eps):
            print("calculated x = ", self.x)
            print("f(x) = ", f(self.x))
        else: 
            # иначе пересчитываем вектор
            self.vector = [self.vector[0]/self.gamma, self.vector[1]/self.gamma]
            self.calculated_vector = calculate_vector(self.vector)
            # продолжаем вычисления
            self.__step_2()
    
    def __step_4(self, calculated_x):
        # осуществляем шаг в направлении убывания
        # осуществляем спуск с ak = 2
        x1 = [self.x[0] + 2 * (calculated_x[0] - self.x[0]), self.x[1] + 2 * (calculated_x[1] - self.x[1])]
        # производим исследующий поиск
        search_alg = ResearchSearch(x1, self.vector)
        search_alg.start()
        # если поиск успешен, то продолжаем
        if f(search_alg.x) < f(calculated_x):
            self.x = calculated_x
            self.__step_4(search_alg.x)
        # иначе переходим на пересчитывание вектора
        else: self.__step_3()

In [9]:
x0 = [-0.2, 2]
eps = 0.0005
vector = [1, 1]
gamma = 1.5
alg = HookJeeves(x0, vector, eps, gamma).start()

calculated x =  [0.4564805688426046, -0.11491720821197966]
f(x) =  -36.324200049259865


In [5]:
class SymplexBuilder: 
    def __init__(self, x_1_1, n, l) -> None:
        self.x_start = x_1_1
        self.n = n
        self.length = l

    # вычисление вершины симплекса S1 по формуле (6.1)
    def decider(self, result, i, j):
        if i == j+1:
            return result[0][j-1] + ((np.sqrt(self.n+1) - 1)/(self.n*np.sqrt(2))*self.length)
        else:
            return result[0][j-1] + ((np.sqrt(self.n+1) + self.n - 1)/(self.n*np.sqrt(2))*self.length)
    
    # функция построения симплекса
    def build(self):
        # инициализируем массив [[0,0], ...]
        result= np.zeros((self.n+1, self.n), dtype='float').tolist()
        # x_1_1 = начальная точка
        result[0] = self.x_start
        for i in range(2, self.n+2):
            for j in range(1, self.n+1):
                # вычисляем остальные вершины симплекса
                result[i-1][j-1] = self.decider(result, i,j)
        # возвращаем построенный симплекс
        return result        
        

In [6]:
class SymplexMinimization:
    def __init__(self, x_start: list, func, eps=0.01, sigma=0.5, length = 0.5, min_length = 0.001) -> None:
        self.x_start = x_start
        self.eps = eps
        self.sigma = sigma
        self.func = func
        self.n = len(x_start)
        self.l = length
        # строим начальный симплекс по начальной точке, N, длину ребра L
        self.symplex = SymplexBuilder(self.x_start, self.n, self.l).build()
        self.f0 = f(self.x_start)

    def step_1(self):
        # сортируем симплекса по f(x1, x2)
        self.symplex = sorted(self.symplex, key=lambda x: self.func(x))
        self.step_2()
    
    # 1/n+1 * сумма от i = 1 до n + 1 выражения (f(x^k,i) - f(x^k))^2     
    def sum_func(self, symplex):
        sum = 0.0
        for i in symplex:
            sum += (f(i) - f(symplex[0]))**2
        return 1.0/(self.n + 1) * sum

    # неравенство для остановки вычислений
    def stop_criteria(self):
        return self.sum_func(self.symplex) < (self.eps**2)
    
    def step_2(self): 
         # если выполнено условие остановки
         if self.stop_criteria():
             # осуществляем вывод
             print('last symplex', self.symplex)
             print('x*=', self.symplex[0])
             print('f(x*)=', self.func(self.symplex[0]))
             return self.symplex[0]
         # иначе продолжаем вычисления
         else: self.step_3()

    # функция вычисления равноудалённой точки Xc
    def equidistant_point(self, symplex):
        sum:list[float] = [0.0, 0.0]
        for i in range(self.n):
            sum[0] += symplex[i][0]
            sum[1] += symplex[i][1]
        return [float(2.0/self.n) * sum[0], float(2.0/self.n) * sum[1]]
    
    def step_3(self):
        # создаём кандидата (отражённую вершину)
        candidate = [0.0, 0.0]
        # вычисляем равноудалённую точку
        xc = self.equidistant_point(self.symplex)
        # производим отражение
        candidate[0] = xc[0] - self.symplex[-1][0]
        candidate[1] = xc[1] - self.symplex[-1][1]
        # если не выполняем неравенство для редукции
        if f(candidate) < f(self.symplex[-1]):
            # то продолжаем отражать
            self.symplex[-1] = candidate
            self.step_1()
        else:
            # иначе осуществляем редукцию
            self.step_4()
    
    def step_4(self):
        for i in range(1, self.n+1):
            # находим вершины нового симплекса Si
            self.symplex[i][0] = self.symplex[0][0] + self.sigma * (self.symplex[i][0] - self.symplex[0][0])
            self.symplex[i][1] = self.symplex[0][1] + self.sigma * (self.symplex[i][1] - self.symplex[0][1])
        self.step_1()

    # функция для старта минимизации
    def minimize(self):
        self.step_1()

In [17]:
class Nelder_Mead:
    def __init__(self, func, eps=0.01, alpha=0.5, beta = 1.0, gamma = 2.0) -> None:
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.func = func
        self.eps = eps
        self.n = 2
        # начальный симплекс
        self.symplex = [[0.0, 0.0], [1.0, 0.0], [0.0, 1.0]]
        # максимальное число итераций
        self.max_iter = 5000
        # номер итерации
        self.i = 1

    def step_2(self):
        # сортируем симплекс по f(x1, x2)
        self.symplex = sorted(self.symplex, key=lambda x: self.func(x))
        # определяем  точки лучшую (минимальное f), хорошую, худшую(максимальное f)
        # для текущего симплекса 
        self.b = self.symplex[0]
        self.g = self.symplex[1]
        self.w = self.symplex[2]
        # вычисляем определяем середину отрезка для худшей и хорошей точек
        self._mid = self.mid(self.g, self.b)

    # отражение
    def step_3(self):
        self.xr = [0.0, 0.0]
        self.xr[0] = self._mid[0] + self.alpha * (self._mid[0] - self.w[0])
        self.xr[1] = self._mid[1] + self.alpha * (self._mid[1] - self.w[1])
        if self.func(self.xr) < self.func(self.g):
            self.w = self.xr.copy()
        else:
            if self.func(self.xr) < self.func(self.w):
                self.w = self.xr.copy()
            c = self.mid(self.w, self._mid)
            if self.func(c) < self.func(self.w):
                self.w = c.copy()
    
    # расширение
    def step_4(self):
        if self.func(self.xr) < self.func(self.b):
            xe = [0.0, 0.0]
            xe[0]= self._mid[0] + self.gamma * (self.xr[0] - self._mid[0])
            xe[1]= self._mid[1] + self.gamma * (self.xr[1] - self._mid[1])
            if self.func(xe) < self.func(self.xr):
                self.w = xe.copy()
            else: self.w = self.xr.copy()
    
    # сокращение
    def step_5(self):
        if self.func(self.xr) > self.func(self.g):
            xc = [0.0, 0.0]
            xc[0] = self._mid[0] + self.beta * (self.w[0] - self._mid[0])
            xc[1] = self._mid[1] + self.beta * (self.w[1] - self._mid[1])
        self.symplex[0] = self.w.copy()
        self.symplex[1] = self.g.copy()
        self.symplex[2] = self.b.copy()
        self.i+=1

    # функция вычисления середины отрезка AB
    def mid(self, a, b):
        return [(a[0] + b[0])/2.0, (a[1] + b[1])/2.0]
    
    # 1/n+1 * сумма от i = 1 до n + 1 выражения (f(x^k,i) - f(x^k))^2     
    def sum_func(self, symplex):
        sum = 0.0
        for i in symplex:
            sum += (f(i) - f(symplex[0]))**2
        return 1.0/(self.n + 1) * sum

    # условие остановы
    def stop_criteria(self):
        return self.sum_func(self.symplex) < (self.eps**2)
   

    def minimize(self):
        # пока не произошло максимальное кол-во итераций или встречена необходимая точность
        while(not(self.i == self.max_iter or self.stop_criteria())):
            self.step_2()
            self.step_3()
            self.step_4()
            self.step_5()
        print('last symplex', self.symplex)
        print('x*=', self.symplex[0])
        print('f(x*)=', self.func(self.symplex[0]))

In [279]:
x_start = [0.0, 0.0]
builder = SymplexBuilder(x_start, 2, 2)
builder.build()

[[0.0, 0.0],
 [0.5176380902050414, 1.9318516525781364],
 [1.9318516525781364, 0.5176380902050414]]

In [82]:
def f(x): 
    return 6*x[0]*x[0]-4*x[0]*x[1]+3*x[1]*x[1]+4*np.sqrt(5)*(x[0]+2*x[1])+22



In [83]:
SymplexMinimization([-2.0, 1.0], func=f, eps=0.01, sigma=0.5).minimize()

last symplex [[-2.2409424497865666, -4.5000491348615075], [-2.224766259467659, -4.439678770718441], [-2.2851366236107262, -4.455854961037348]]
x*= [-2.2409424497865666, -4.5000491348615075]
f(x*)= -27.998064248376238


In [7]:
def f(x: list):
    return 105*(x[0]**2) - 2*x[0]*x[1] + 105*(x[1]**2) - 96*x[0] + 25*x[1] - 13
x0 = [-13, 0]
eps = 0.005

SymplexMinimization(x0, func=f, eps=eps, sigma=0.5).minimize()

last symplex [[0.4529242395079657, -0.11468712450893288], [0.460470535025849, -0.11266510071906943], [0.45844851123598557, -0.12021139623695279]]
x*= [0.4529242395079657, -0.11468712450893288]
f(x*)= -36.32319810567925


In [18]:
Nelder_Mead(func=f, eps=0.005).minimize()

last symplex [[0.46161123272031546, -0.11190069653093815], [0.4603600725531578, -0.11743177473545074], [0.4588354825973511, -0.11197590827941895]]
x*= [0.46161123272031546, -0.11190069653093815]
f(x*)= -36.320183431431694


In [14]:
class BitWiseSearch: 
    def __init__(self, x0, step, e, func, static_coord=None, is_y = False) -> None:
        self.e = e
        self.func = func
        self.x0 = x0
        self.h = step
        self.forward = True
        self.static_coord = static_coord
        self.is_second = is_y
    
    def step_2(self):
        self.step_3(self.calculate_step())
    
    # вычисляем следующее значение x + h
    def calculate_step(self):
        # случай, если ищем x для f(x)
        if self.static_coord == None:
            return self.x0+self.h
        # случай, если ищем x1 или x2 для f(x1, x2)
        else: 
            # если зафиксирована x1, то шагаем по x2
            if self.static_coord == 0: 
                return [self.x0[0], self.x0[1]+self.h]
            # иначе шагаем по x1, с фиксацией x2
            else: return [self.x0[0]+self.h, self.x0[1]]
    
    # функция вычисления f от текущего x
    def calculate_f(self, x):
        # для случая, когда f(x)
        if self.static_coord == None:
            return self.func(x)
        # случай, когда у нас f(x1, x2)
        else:
            if self.static_coord == 0: 
                return self.func([self.x0[0], x[1]])
            else: return self.func([x[0], self.x0[1]])

    def step_3(self, x1):
        # проверяем значение f(x0) и f(x1)
        if ((self.forward and self.calculate_f(self.x0)>self.calculate_f(x1)) or (not self.forward and self.calculate_f(x1)<self.calculate_f(self.x0))):
            #если перебор можно осуществлять в текущем направлении, то перемещаемся к новому x1
            self.x0 = x1
            self.step_2()
        else:
            #иначе переходим на шаг 4
            self.step_4(x1)

    def step_4(self, x1 = 0):
        # если шаг по модулю < eps
        if abs(self.h) < self.e:
                # то минимум найден, поиск окончен
                self.x_min = self.x0
                self.f_min = self.func(self.x0)
        # иначе меняем направление и продолжаем перебор в новом направлении
        else:
            self.x0 = x1
            self.h = (-self.h/4.0)
            self.forward = not self.forward
            self.step_2()
    # функция инициализации поиска
    def search(self):
        self.step_2()
        # функция возврата результатов поиска для случая f(x) и f(x1, x2)
        if self.static_coord != None:
            if self.is_second == True:
                return [self.x_min[0], self.x_min[1]]
            else: 
                return [self.x_min[0], self.x_min[1]]
        else:
            return self.x_min

        


In [ ]:
BitWiseSearch(0, 0.25, 0.1, lambda x: x**4 + np.e**(-x)).search()

0.5

In [15]:
class CyclicSearch:
    def __init__(self, x0:list[float], eps_1, eps_2, eps_optim, func,optim_step:float = 0.25) -> None:
        self.x0 = x0
        self.eps_1 = eps_1
        self.eps_2 = eps_2
        self.eps_optim = eps_optim
        self.func = func
        self.optim_step = optim_step
    # расстояние между двумя точками < eps1
    def criteria_by_path(self, x_prev, x_new):
        return np.sqrt((abs(x_prev[0]-x_new[0]))+(abs(x_prev[1]-x_new[1]))) < self.eps_1
    
    # | f(x0) - f(x1) | < eps2 
    def criteria_by_func(self, x_prev, x_new):
        return abs(self.func(x_prev) - self.func(x_new)) < self.eps_2
    
    def step_1(self):
        # полагаем, что j =1
        self.j = 1
        # фиксируем текущую точку
        self.x_new = self.x0.copy()
        # переходим к шагу два
        self.step_2()

    # функция для определения координаты для поразрядного поиска
    def is_y(self, j):
        if j == 1:
            return False
        else: return True

    # функция для определения индекса точки для поразрядного поиска
    def get_coord(self):
        if self.j == 1:
            return 1
        else: return 0
    
    def step_2(self):
        # производим поразрядный поиск для j компоненты точки
        self.x_new = BitWiseSearch(self.x_new, self.optim_step, self.eps_optim, self.func, self.get_coord(), self.is_y(self.j)).search()
        print('current iter x_new', self.x_new)
        print('current iter f(x_new)', self.func(self.x_new))
        self.step_3()

    def step_3(self):
        # если j < 2, то переходим к шагу 2
        if self.j != 2:
            self.j += 1
            self.step_2()
        # иначе к шагу 4
        else: self.step_4()
    
    def step_4(self):
        # проверяется условие остановки по точности и расстоянию между новым минимумом и предыдущим
        if (self.criteria_by_path(self.x0, self.x_new) or self.criteria_by_func(self.x0, self.x_new)):
            # записываем ответ в x0
            self.x0 = self.x_new.copy()
            # выводим результат
            print('final result-> x', self.x0)
            print('final refulst-> f(x)', self.func(self.x0))
            return self.x0
        else: 
            # иначе записываем в x0 результат текущей итерации
            self.x0 = self.x_new
            # и продолжаем поиск
            self.j = 1
            self.step_2()

    def start(self):
        self.step_1()

In [22]:
def f(x: list):
    return 105*(x[0]**2) - 2*x[0]*x[1] + 105*(x[1]**2) - 96*x[0] + 25*x[1] - 13
x0 = [-13, 0]
eps = 0.005

CyclicSearch(x0, eps, eps, 0.001, f, 0.25).start()

current iter x_new [0.45703125, 0]
current iter f(x_new) -34.94285583496094
current iter x_new [0.45703125, -0.1142578125]
current iter f(x_new) -36.324103355407715
current iter x_new [0.4560546875, -0.1142578125]
current iter f(x_new) -36.32420349121094
current iter x_new [0.4560546875, -0.1142578125]
current iter f(x_new) -36.32420349121094
final result-> x [0.4560546875, -0.1142578125]
final refulst-> f(x) -36.32420349121094
